# LLaDA: 拡散言語モデル学習ノートブック

このノートブックでは、LLaDA（Large Language Diffusion with mAsking）を使って拡散言語モデルの特徴と動作原理を学習します。

## 📚 目次
1. [拡散言語モデルとは？](#1-拡散言語モデルとは)
2. [LLaDAの特徴](#2-lladaの特徴)
3. [環境設定（Google Colab）](#3-環境設定google-colab)
4. [基本的な生成実験](#4-基本的な生成実験)
5. [拡散プロセスの可視化](#5-拡散プロセスの可視化)
6. [パラメータの影響を調べる](#6-パラメータの影響を調べる)
7. [自己回帰モデルとの比較](#7-自己回帰モデルとの比較)
8. [まとめと発展的な実験](#8-まとめと発展的な実験)

## 1. 拡散言語モデルとは？

### 従来の自己回帰言語モデル
- **GPT**や**LLaMA**のような自己回帰モデルは、テキストを**左から右**へ順番に生成します
- 各ステップで次のトークンを予測し、それを元に次の予測を行います

```
自己回帰: "The" → "The cat" → "The cat sits" → "The cat sits on"
```

### 拡散言語モデル
- **拡散モデル**は画像生成（DALL-E、Stable Diffusionなど）で広く使われている技術です
- **LLaDA**は拡散の概念をテキスト生成に応用した革新的なモデルです

#### 拡散プロセスの特徴：
1. **全体から部分へ**: 最初に全体の構造を決め、徐々に詳細を埋めていく
2. **並列処理**: すべての位置を同時に考慮できる
3. **反復改善**: 複数のステップを通じて品質を向上させる

```
拡散: "[MASK] [MASK] [MASK] [MASK]" → "The [MASK] [MASK] on" → "The cat [MASK] on" → "The cat sits on"
```

## 2. LLaDAの特徴

### 🔑 主要な概念

#### マスクトークン [MASK]
- トークンID: `126336`
- 未確定の位置を表す特別なトークン
- 生成過程で段階的に実際のトークンに置き換えられる

#### 信頼度ベースの選択
- モデルは各位置でのトークン予測に**信頼度スコア**を付ける
- 高信頼度のトークンから順番に確定していく
- 低信頼度のトークンは再マスクされ、次のステップで再予測

#### 双方向注意
- 自己回帰モデルと違い、**因果マスキングがない**
- 各トークンが文章全体の文脈を参照できる
- より一貫性のある文章生成が可能

### 🏗️ アーキテクチャ
- **ベースモデル**: Transformer Encoder（8Bパラメータ）
- **学習データ**: 2.3Tトークン
- **2つのバリアント**:
  - `LLaDA-8B-Base`: 事前学習のみ
  - `LLaDA-8B-Instruct`: 指示に従うよう微調整

## 3. 環境設定（Google Colab）

Google Colabで実行する場合は、以下のセルを実行してください：

In [ ]:
# Google Colabでの実行用
import sys

# Google Colabかどうかをチェック
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Google Colab環境を検出しました")
    
    # 必要なライブラリをインストール
    !pip install transformers==4.49.0 accelerate==0.34.2 torch numpy matplotlib ipywidgets
    
    # GPUが利用可能かチェック
    import torch
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("ローカル環境で実行中です")

In [ ]:
# 必要なライブラリをインポート
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import time
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定（Colab用）
if IN_COLAB:
    !apt-get -qq install fonts-noto-cjk
    import matplotlib.font_manager as fm
    plt.rcParams['font.family'] = 'DejaVu Sans'

print("ライブラリのインポートが完了しました")

## 4. 基本的な生成実験

まず、LLaDAの基本的な使い方を学びましょう。

In [ ]:
# LLaDAモデルとトークナイザーの読み込み
print("LLaDAモデルを読み込み中...（数分かかります）")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用デバイス: {device}")

# モデルとトークナイザーの読み込み
model_name = 'GSAI-ML/LLaDA-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16
).to(device).eval()

print("モデルの読み込みが完了しました！")
print(f"モデルサイズ: {sum(p.numel() for p in model.parameters()) / 1e9:.1f}B パラメータ")

### LLaDAの核となる関数を実装

ここでは、LLaDAの拡散プロセスを理解するために、重要な関数を詳しく見ていきます。

In [ ]:
# 重要な定数
MASK_ID = 126336  # [MASK]トークンのID

def add_gumbel_noise(logits, temperature):
    """
    Gumbelノイズを追加してサンプリングを行う
    temperature=0: 決定論的（最も確率の高いトークンを選択）
    temperature>0: ランダム性を導入（高い温度ほどランダム）
    """
    if temperature == 0:
        return logits
    
    # 高精度演算のためfloat64を使用
    logits = logits.to(torch.float64)
    noise = torch.rand_like(logits, dtype=torch.float64)
    gumbel_noise = (-torch.log(noise)) ** temperature
    return logits.exp() / gumbel_noise

def get_num_transfer_tokens(mask_index, steps):
    """
    各ステップで何個のトークンをマスク解除するかを計算
    線形スケジュールに従って均等に分散
    """
    mask_num = mask_index.sum(dim=1, keepdim=True)  # マスクされたトークンの総数
    
    base = mask_num // steps  # 基本的な数
    remainder = mask_num % steps  # 余り
    
    # 各ステップでのトークン数を計算
    num_transfer_tokens = torch.zeros(
        mask_num.size(0), steps, 
        device=mask_index.device, 
        dtype=torch.int64
    ) + base
    
    # 余りを最初のステップに分散
    for i in range(mask_num.size(0)):
        num_transfer_tokens[i, :remainder[i]] += 1
    
    return num_transfer_tokens

print("コア関数の定義が完了しました")

### 簡単な生成実験

In [ ]:
@torch.no_grad()
def generate_llada(model, prompt, steps=32, gen_length=64, block_length=32, 
                   temperature=0.0, cfg_scale=0.0, remasking='low_confidence'):
    """
    完全なLLaDA生成関数（Google Colab単体動作用）
    元のgenerate.pyの完全実装
    """
    # プロンプトをトークン化
    if isinstance(prompt, str):
        # Instructモデル用のチャットテンプレートを適用
        messages = [{"role": "user", "content": prompt}]
        formatted_prompt = tokenizer.apply_chat_template(
            messages, add_generation_prompt=True, tokenize=False
        )
        input_ids = tokenizer(formatted_prompt)['input_ids']
        input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)
    else:
        input_ids = prompt
    
    prompt_length = input_ids.shape[1]
    
    # 応答部分を[MASK]で初期化
    x = torch.full(
        (1, prompt_length + gen_length), 
        MASK_ID, 
        dtype=torch.long
    ).to(device)
    x[:, :prompt_length] = input_ids.clone()
    
    # プロンプト部分のインデックス（CFG用）
    prompt_index = (x != MASK_ID)
    
    # ブロック処理の設定（修正版：より柔軟な設定）
    # gen_lengthがblock_lengthで割り切れない場合も対応
    block_length = min(block_length, gen_length)
    num_blocks = (gen_length + block_length - 1) // block_length  # 切り上げ除算
    
    # stepsがnum_blocksで割り切れない場合の対応
    if steps % num_blocks != 0:
        # 最も近い割り切れるsteps数に調整
        adjusted_steps = ((steps + num_blocks - 1) // num_blocks) * num_blocks
        print(f"ステップ数を {steps} から {adjusted_steps} に調整しました（{num_blocks}ブロックに対応）")
        steps = adjusted_steps
    
    steps_per_block = steps // num_blocks
    
    # ステップごとの生成記録
    generation_history = []
    
    print(f"生成開始: {num_blocks}ブロック x {steps_per_block}ステップ = 計{steps}ステップ")
    print(f"ブロック設定: gen_length={gen_length}, block_length={block_length}")
    
    # 各ブロックを処理
    for block_idx in range(num_blocks):
        block_start = prompt_length + block_idx * block_length
        block_end = min(prompt_length + (block_idx + 1) * block_length, x.shape[1])
        
        print(f"\\nブロック {block_idx + 1}/{num_blocks} (位置 {block_start}-{block_end-1}):")
        
        # 現在のブロックのマスクインデックス
        block_mask_index = (x[:, block_start:block_end] == MASK_ID)
        
        # スキップ条件: ブロックにマスクがない場合
        if not block_mask_index.any():
            print(f"  ブロック {block_idx + 1}: マスクなし、スキップ")
            continue
            
        # このブロックでのトークン転送数を計算
        num_transfer_tokens = get_num_transfer_tokens(block_mask_index, steps_per_block)
        
        # ブロック内の各ステップ
        for step in range(steps_per_block):
            # 現在のマスク位置を取得
            mask_index = (x == MASK_ID)
            
            if not mask_index.any():
                print(f"  ステップ {step + 1}: 全トークン確定済み")
                break
            
            # 分類器フリーガイダンス (CFG)
            if cfg_scale > 0.0:
                # 無条件生成用：プロンプト部分もマスク
                un_x = x.clone()
                un_x[prompt_index] = MASK_ID
                x_ = torch.cat([x, un_x], dim=0)
                logits = model(x_).logits
                logits, un_logits = torch.chunk(logits, 2, dim=0)
                # CFGスケールを適用
                logits = un_logits + (cfg_scale + 1) * (logits - un_logits)
            else:
                logits = model(x).logits
            
            # Gumbelノイズでサンプリング
            logits_with_noise = add_gumbel_noise(logits, temperature)
            x0 = torch.argmax(logits_with_noise, dim=-1)
            
            # 再マスキング戦略に基づく信頼度計算
            if remasking == 'low_confidence':
                p = F.softmax(logits, dim=-1)
                x0_p = torch.squeeze(
                    torch.gather(p, dim=-1, index=torch.unsqueeze(x0, -1)), -1
                )
            elif remasking == 'random':
                x0_p = torch.rand((x0.shape[0], x0.shape[1]), device=x0.device)
            else:
                raise NotImplementedError(f"Remasking strategy '{remasking}' not implemented")
            
            # 現在のブロック以外の位置は除外
            x0_p[:, block_end:] = -float('inf')
            
            # マスク位置のみの予測と信頼度
            x0 = torch.where(mask_index, x0, x)
            confidence = torch.where(mask_index, x0_p, -float('inf'))
            
            # 高信頼度のトークンを選択してマスク解除
            transfer_index = torch.zeros_like(x0, dtype=torch.bool, device=device)
            for j in range(confidence.shape[0]):
                num_tokens_to_transfer = min(
                    num_transfer_tokens[j, step].item(),
                    mask_index[j].sum().item()  # 実際のマスク数を超えないよう制限
                )
                if num_tokens_to_transfer > 0:
                    _, select_index = torch.topk(confidence[j], k=num_tokens_to_transfer)
                    transfer_index[j, select_index] = True
            
            # 選択されたトークンを確定
            x[transfer_index] = x0[transfer_index]
            
            # 進捗を記録
            current_text = tokenizer.decode(
                x[0, prompt_length:], skip_special_tokens=False
            )
            avg_conf = confidence[0, transfer_index[0]].mean().item() if transfer_index[0].any() else 0
            
            generation_history.append({
                'block': block_idx,
                'step': step,
                'global_step': block_idx * steps_per_block + step,
                'text': current_text,
                'num_revealed': transfer_index[0].sum().item(),
                'avg_confidence': avg_conf
            })
            
            print(f"  ステップ {step+1:2d}: {transfer_index[0].sum().item():2d}個確定 "
                  f"(信頼度: {avg_conf:.3f})")
    
    # 最終結果
    final_text = tokenizer.decode(
        x[0, prompt_length:], skip_special_tokens=True
    )
    
    return final_text, generation_history

# 簡略版も保持（学習用）
@torch.no_grad()
def simple_generate(model, prompt, steps=32, gen_length=64, temperature=0.0):
    """
    簡略化されたLLaDA生成関数（学習用）
    """
    return generate_llada(
        model, prompt, 
        steps=steps, gen_length=gen_length, block_length=gen_length,
        temperature=temperature, cfg_scale=0.0, remasking='low_confidence'
    )

# テスト実行
test_prompt = "What is the capital of Japan?"
print(f"\\n質問: {test_prompt}")
print("=" * 50)

result, history = simple_generate(
    model, test_prompt, 
    steps=16, gen_length=32, temperature=0.0
)

print("=" * 50)
print(f"最終回答: {result}")

## 5. 拡散プロセスの可視化

LLaDAの拡散プロセスがどのように動作するかを可視化してみましょう。

In [ ]:
def visualize_generation_process(history, title="LLaDA Generation Process"):
    """
    生成プロセスを段階的に表示
    """
    print(f"\n{title}")
    print("=" * 60)
    
    for entry in history:
        step = entry['step']
        text = entry['text']
        num_revealed = entry['num_revealed']
        avg_conf = entry['avg_confidence']
        
        # [MASK]を視覚的に表現
        visual_text = text.replace('<|reserved_special_token_250|>', '[MASK]')
        
        print(f"Step {step+1:2d} (+{num_revealed:2d} tokens, conf={avg_conf:.3f}):")
        print(f"  {visual_text}")
        print()

def detect_token_changes(prev_tokens, curr_tokens):
    """
    前のステップと現在のステップでのトークン変化を検出
    """
    changes = []
    
    # トークン化して比較
    if prev_tokens is None:
        # 初回の場合、すべてが新規
        for i, token in enumerate(curr_tokens):
            if token != '[MASK]':
                changes.append((i, 'new', token))
    else:
        # 前のステップと比較
        for i, (prev_token, curr_token) in enumerate(zip(prev_tokens, curr_tokens)):
            if prev_token != curr_token:
                if prev_token == '[MASK]' and curr_token != '[MASK]':
                    changes.append((i, 'revealed', curr_token))
                elif prev_token != '[MASK]' and curr_token == '[MASK]':
                    changes.append((i, 'masked', prev_token))
                elif prev_token != '[MASK]' and curr_token != '[MASK]':
                    changes.append((i, 'changed', curr_token))
    
    return changes

def get_confidence_color(confidence):
    """
    信頼度に基づいて落ち着いた色を決定
    """
    if confidence >= 0.8:
        return '\033[38;5;28m'  # 深い緑（高信頼度）
    elif confidence >= 0.6:
        return '\033[38;5;94m'  # オリーブ（中信頼度）
    elif confidence >= 0.4:
        return '\033[38;5;130m'  # ブラウン（低信頼度）
    else:
        return '\033[38;5;240m'  # グレー（非常に低信頼度）

def visualize_generation_with_highlights(history, title="LLaDA Generation Process (Enhanced)", show_details=True):
    """
    新しく生成された単語をハイライト表示する拡張可視化関数（落ち着いた配色版）
    """
    print(f"\n{title}")
    print("=" * 70)
    
    # 色の説明
    if show_details:
        print("信頼度色分け:")
        print("  \033[38;5;28m■\033[0m 高信頼度 (≥0.8)  \033[38;5;94m■\033[0m 中信頼度 (≥0.6)  \033[38;5;130m■\033[0m 低信頼度 (≥0.4)  \033[38;5;240m■\033[0m 極低 (<0.4)")
        print("  強調: \033[1m太字\033[0m = 新規生成  \033[2m薄字\033[0m = 再マスク  \033[4m下線\033[0m = 変更")
        print()
    
    prev_tokens = None
    
    for i, entry in enumerate(history):
        global_step = entry.get('global_step', entry['step'])
        block = entry.get('block', 0)
        step = entry['step']
        text = entry['text']
        num_revealed = entry['num_revealed']
        avg_conf = entry['avg_confidence']
        
        # [MASK]を標準化
        clean_text = text.replace('<|reserved_special_token_250|>', '[MASK]')
        
        # トークンに分割（簡易版）
        curr_tokens = clean_text.split()
        
        # 変化を検出
        changes = detect_token_changes(prev_tokens, curr_tokens)
        
        # ステップ情報を表示
        block_info = f"[Block {block+1}]" if 'block' in entry else ""
        print(f"Step {global_step+1:2d} {block_info} (+{num_revealed:2d} tokens, conf={avg_conf:.3f}):")
        
        # ハイライト付きテキストを構築
        highlighted_text = []
        change_map = {pos: (change_type, token) for pos, change_type, token in changes}
        
        for pos, token in enumerate(curr_tokens):
            if pos in change_map:
                change_type, _ = change_map[pos]
                color = get_confidence_color(avg_conf)
                
                if change_type == 'revealed':
                    # 新規生成: 太字 + 色
                    highlighted_text.append(f"{color}\033[1m{token}\033[0m")
                elif change_type == 'masked':
                    # 再マスク: 薄字
                    highlighted_text.append(f"\033[2m[MASK]\033[0m")
                elif change_type == 'changed':
                    # 変更: 下線 + 色
                    highlighted_text.append(f"{color}\033[4m{token}\033[0m")
                else:
                    highlighted_text.append(f"{color}{token}\033[0m")
            else:
                # 変化なし
                if token == '[MASK]':
                    highlighted_text.append(f"\033[38;5;245m{token}\033[0m")  # 薄いグレー
                else:
                    highlighted_text.append(token)
        
        # 表示
        print(f"  {' '.join(highlighted_text)}")
        
        # 変化の詳細を表示
        if show_details and changes:
            change_summary = []
            for pos, change_type, token in changes:
                if change_type == 'revealed':
                    change_summary.append(f"pos{pos}:新規({token})")
                elif change_type == 'masked':
                    change_summary.append(f"pos{pos}:マスク")
                elif change_type == 'changed':
                    change_summary.append(f"pos{pos}:変更({token})")
            
            if change_summary:
                print(f"    変化: {', '.join(change_summary)}")
        
        print()
        prev_tokens = curr_tokens

def visualize_step_by_step_animation(history, title="LLaDA Step-by-Step Animation", delay=1.0):
    """
    段階的なアニメーション風表示（Jupyter用）
    """
    import time
    try:
        from IPython.display import display, clear_output
        use_jupyter = True
    except ImportError:
        use_jupyter = False
    
    print(f"{title}")
    print("=" * 50)
    
    for i, entry in enumerate(history):
        if use_jupyter:
            clear_output(wait=True)
        
        # 現在のステップまでを表示
        print(f"{title} - Step {i+1}/{len(history)}")
        print("=" * 50)
        
        visualize_generation_with_highlights(history[:i+1], 
                                           title=f"Progress: {i+1}/{len(history)} steps", 
                                           show_details=False)
        
        if i < len(history) - 1:  # 最後でなければ待機
            time.sleep(delay)

def visualize_block_progression(history):
    """
    ブロック別の生成進捗を可視化（修正版）
    """
    if not history:
        print("履歴データがありません")
        return
        
    print("\nブロック別生成進捗:")
    print("=" * 50)
    
    current_block = -1
    for entry in history:
        block = entry.get('block', 0)
        if block != current_block:
            current_block = block
            print(f"\nBlock {current_block + 1}:")
            
        step = entry['step']
        text = entry['text']
        num_revealed = entry['num_revealed']
        avg_conf = entry['avg_confidence']
        
        # [MASK]を視覚的に表現
        visual_text = text.replace('<|reserved_special_token_250|>', '[MASK]')
        
        print(f"  Step {step+1:2d}: +{num_revealed:2d} tokens (conf={avg_conf:.3f})")
        print(f"    状態: {visual_text[:80]}{'...' if len(visual_text) > 80 else ''}")

def compare_generation_steps(history_list, titles=None, highlight_differences=True):
    """
    複数の生成履歴を並べて比較表示
    """
    if titles is None:
        titles = [f"Generation {i+1}" for i in range(len(history_list))]
    
    print("生成プロセス比較")
    print("=" * 80)
    
    max_steps = max(len(history) for history in history_list)
    
    for step in range(max_steps):
        print(f"\nStep {step+1}:")
        print("-" * 60)
        
        for i, (history, title) in enumerate(zip(history_list, titles)):
            if step < len(history):
                entry = history[step]
                text = entry['text'].replace('<|reserved_special_token_250|>', '[MASK]')
                conf = entry['avg_confidence']
                revealed = entry['num_revealed']
                
                print(f"  {title}: (+{revealed:2d}, conf={conf:.3f})")
                print(f"    {text}")
            else:
                print(f"  {title}: [完了]")
        print()

# 前回の結果を拡張可視化
print("拡張可視化機能のデモ:")
if 'history' in locals():
    visualize_generation_with_highlights(history, "日本の首都に関する質問の生成プロセス（拡張版）")
else:
    print("履歴データが見つかりません。先に生成実験を実行してください。")

### より複雑な質問での実験

In [ ]:
# より複雑な数学の問題を試してみる
math_prompt = "Solve this step by step: If a train travels 120 km in 1.5 hours, what is its average speed in km/h?"

print(f"数学問題: {math_prompt}")
print("=" * 70)

math_result, math_history = simple_generate(
    model, math_prompt,
    steps=24, gen_length=80, temperature=0.0
)

print("=" * 70)
print(f"回答: {math_result}")

# 新機能：拡張可視化でプロセスを詳細に表示
print("\n" + "=" * 40)
print("新機能デモ: ハイライト付き可視化")
print("=" * 40)

visualize_generation_with_highlights(math_history[-8:], "数学問題の詳細生成プロセス（最後の8ステップ）")

# アニメーション風表示のデモ
print("\n" + "=" * 40)
print("新機能デモ: 段階的アニメーション表示")
print("=" * 40)
print("注意: Jupyter環境では画面がクリアされてアニメーションのように表示されます")

# 最後の5ステップをアニメーション風に表示
visualize_step_by_step_animation(math_history[-5:], 
                                title="数学問題生成の最終段階", 
                                delay=0.5)

# 従来の表示と比較
print("\n" + "=" * 40)
print("従来表示 vs 拡張表示の比較")
print("=" * 40)

print("従来の表示:")
visualize_generation_process(math_history[-3:], "従来の可視化")

print("\n拡張表示（ハイライト付き）:")
visualize_generation_with_highlights(math_history[-3:], "拡張可視化", show_details=True)

## 6. パラメータの影響を調べる

LLaDAの異なるパラメータが生成結果にどのような影響を与えるかを実験してみましょう。

### 6.1 ステップ数の影響

In [ ]:
def compare_different_steps(prompt, step_counts=[8, 16, 32]):
    """
    異なるステップ数での生成結果を比較（修正版）
    """
    print(f"プロンプト: {prompt}")
    print("=" * 60)
    
    results = {}
    
    for steps in step_counts:
        print(f"\\n📊 {steps}ステップでの生成:")
        start_time = time.time()
        
        # ブロック設定を調整：ステップ数に応じて適切なブロックサイズを選択
        if steps <= 16:
            block_length = 48  # 1ブロックで処理
        else:
            block_length = 24  # 2ブロックで処理
        
        result, history = generate_llada(
            model, prompt,
            steps=steps, gen_length=48, block_length=block_length,
            temperature=0.0, cfg_scale=0.0, remasking='low_confidence'
        )
        
        end_time = time.time()
        
        results[steps] = {
            'text': result,
            'time': end_time - start_time,
            'history': history
        }
        
        print(f"結果: {result}")
        print(f"時間: {end_time - start_time:.2f}秒")
    
    return results

def compare_advanced_settings(prompt):
    """
    高度な設定での比較実験（修正版）
    """
    print(f"\\n🔬 高度な設定比較")
    print(f"プロンプト: {prompt}")
    print("=" * 60)
    
    settings = [
        {
            "name": "標準設定",
            "steps": 16, "block_length": 64, "temperature": 0.0, 
            "cfg_scale": 0.0, "remasking": "low_confidence"
        },
        {
            "name": "高品質設定",
            "steps": 32, "block_length": 32, "temperature": 0.0, 
            "cfg_scale": 0.0, "remasking": "low_confidence"
        },
        {
            "name": "創造的設定",
            "steps": 24, "block_length": 32, "temperature": 0.3, 
            "cfg_scale": 0.0, "remasking": "low_confidence"
        },
        {
            "name": "CFG強化設定",
            "steps": 20, "block_length": 32, "temperature": 0.0, 
            "cfg_scale": 1.0, "remasking": "low_confidence"
        },
        {
            "name": "ランダム再マスク",
            "steps": 16, "block_length": 64, "temperature": 0.0, 
            "cfg_scale": 0.0, "remasking": "random"
        }
    ]
    
    results = []
    
    for setting in settings:
        print(f"\\n🧪 {setting['name']}:")
        print(f"  設定: steps={setting['steps']}, block={setting['block_length']}, "
              f"temp={setting['temperature']}, cfg={setting['cfg_scale']}, "
              f"remasking={setting['remasking']}")
        
        start_time = time.time()
        result, history = generate_llada(
            model, prompt,
            steps=setting['steps'],
            gen_length=64,
            block_length=setting['block_length'],
            temperature=setting['temperature'],
            cfg_scale=setting['cfg_scale'],
            remasking=setting['remasking']
        )
        generation_time = time.time() - start_time
        
        avg_confidence = np.mean([h['avg_confidence'] for h in history if h['avg_confidence'] > 0])
        
        results.append({
            'name': setting['name'],
            'result': result,
            'time': generation_time,
            'avg_confidence': avg_confidence,
            'setting': setting
        })
        
        print(f"  結果: {result}")
        print(f"  時間: {generation_time:.2f}秒, 平均信頼度: {avg_confidence:.3f}")
    
    return results

# 創作的なプロンプトで実験
creative_prompt = "Write a haiku about artificial intelligence."
step_results = compare_different_steps(creative_prompt)

# 高度な設定での比較
advanced_results = compare_advanced_settings("Tell me a short story about a time-traveling scientist.")

### 6.2 温度（Temperature）の影響

In [ ]:
def compare_temperatures_with_highlights(prompt, temperatures=[0.0, 0.3, 0.7]):
    """
    異なる温度設定での生成結果を拡張可視化で比較
    """
    print(f"温度比較実験（拡張可視化）")
    print(f"プロンプト: {prompt}")
    print("=" * 60)
    
    all_histories = []
    all_titles = []
    
    for temp in temperatures:
        print(f"\n温度 {temp}での生成:")
        
        result, history = simple_generate(
            model, prompt,
            steps=16, gen_length=40, temperature=temp
        )
        
        print(f"結果: {result}")
        
        # 生成プロセスをハイライト表示
        print(f"\n温度{temp}の生成プロセス:")
        visualize_generation_with_highlights(history[-5:], 
                                           title=f"温度{temp}での生成（最後の5ステップ）", 
                                           show_details=False)
        
        all_histories.append(history[-3:])  # 最後の3ステップを保存
        all_titles.append(f"温度{temp}")
        
        if temp == 0.0:
            print("  → 決定論的（毎回同じ結果）")
        elif temp < 0.5:
            print("  → 低ランダム性（安定した結果）")
        else:
            print("  → 高ランダム性（創造的だが不安定）")
    
    # 複数温度の比較表示
    print(f"\n温度別生成プロセス比較:")
    compare_generation_steps(all_histories, all_titles)

def demonstrate_new_visualization_features():
    """
    新しい可視化機能の包括的なデモンストレーション
    """
    print("LLaDA拡張可視化機能デモンストレーション")
    print("=" * 60)
    
    # デモ用の短い生成実験
    demo_prompt = "What makes a good leader?"
    
    print(f"デモプロンプト: {demo_prompt}")
    print("生成設定: 16ステップ, 48トークン, 温度0.2")
    print("-" * 50)
    
    demo_result, demo_history = generate_llada(
        model, demo_prompt,
        steps=16, gen_length=48, block_length=24,
        temperature=0.2, cfg_scale=0.0, remasking='low_confidence'
    )
    
    print(f"\n生成結果: {demo_result}")
    
    # 機能1: ハイライト付き可視化
    print(f"\n" + "=" * 30)
    print("機能1: ハイライト付き可視化")
    print("=" * 30)
    visualize_generation_with_highlights(demo_history, 
                                       "リーダーシップに関する生成プロセス", 
                                       show_details=True)
    
    # 機能2: アニメーション風表示
    print(f"\n" + "=" * 30)
    print("機能2: アニメーション風表示（最後の6ステップ）")
    print("=" * 30)
    visualize_step_by_step_animation(demo_history[-6:], 
                                   title="生成プロセスのアニメーション", 
                                   delay=0.3)
    
    # 機能3: 信頼度とブロック分析
    print(f"\n" + "=" * 30)
    print("機能3: ブロック別詳細分析")
    print("=" * 30)
    visualize_block_progression(demo_history)
    
    return demo_result, demo_history

# 物語の始まりを拡張可視化で生成
story_prompt = "Once upon a time, in a small village"
compare_temperatures_with_highlights(story_prompt)

# 包括的なデモンストレーション
print("\n" + "=" * 50)
print("新機能の包括的デモンストレーション")
print("=" * 50)

demo_result, demo_history = demonstrate_new_visualization_features()

## 新機能: 拡張可視化システム

このセクションでは、LLaDAの生成プロセスをより深く理解するための拡張可視化機能を紹介します。

### 追加された主要機能

#### 1. **ハイライト付き可視化** (`visualize_generation_with_highlights()`)
- **新しく生成された単語**を太字と色付きで強調表示
- **信頼度レベル**に応じた落ち着いた色分け：
  - 深い緑: 高信頼度 (≥0.8)
  - オリーブ: 中信頼度 (≥0.6)  
  - ブラウン: 低信頼度 (≥0.4)
  - グレー: 極低信頼度 (<0.4)
- **変化の詳細**：位置別の変更履歴を表示

#### 2. **段階的アニメーション表示** (`visualize_step_by_step_animation()`)
- Jupyter環境で**画面をクリア**して動的表示
- 各ステップの進行を**時系列で可視化**
- 調整可能な表示間隔（`delay`パラメータ）

#### 3. **比較表示機能** (`compare_generation_steps()`)
- **複数の生成結果**を並べて比較
- 異なる設定（温度、CFGなど）の影響を分析
- ステップ別の差異を明確化

#### 4. **トークン差分検出** (`detect_token_changes()`)
- 前のステップとの**自動比較**
- 変化タイプの分類：
  - `revealed`: [MASK] → 実際の単語（**太字**表示）
  - `masked`: 実際の単語 → [MASK]（薄字表示）
  - `changed`: 単語 → 別の単語（**下線**表示）

### 使用方法

```python
# 基本的なハイライト表示
visualize_generation_with_highlights(history, "生成プロセス")

# アニメーション風表示（Jupyter推奨）
visualize_step_by_step_animation(history, delay=1.0)

# 複数結果の比較
compare_generation_steps([history1, history2], ["設定A", "設定B"])
```

### 活用のヒント

1. **パラメータ調整時**: 異なる設定での生成プロセスを比較
2. **デバッグ時**: 予期しない生成結果の原因を特定
3. **学習時**: 拡散プロセスの理解を深める
4. **プレゼン時**: 拡散モデルの動作を視覚的に説明

### 実践例

以下のセルでは、これらの新機能を実際に体験できます：

In [ ]:
def analyze_confidence_over_steps(history):
    """
    ステップごとの信頼度の変化をグラフ化
    """
    global_steps = [entry['global_step'] + 1 for entry in history]
    confidences = [entry['avg_confidence'] for entry in history]
    tokens_revealed = [entry['num_revealed'] for entry in history]
    blocks = [entry['block'] for entry in history]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # 信頼度の変化（ブロック別に色分け）
    unique_blocks = sorted(set(blocks))
    colors = plt.cm.Set1(np.linspace(0, 1, len(unique_blocks)))
    
    for i, block in enumerate(unique_blocks):
        block_mask = [b == block for b in blocks]
        block_steps = [s for s, m in zip(global_steps, block_mask) if m]
        block_confs = [c for c, m in zip(confidences, block_mask) if m]
        
        ax1.plot(block_steps, block_confs, 'o-', color=colors[i], 
                linewidth=2, markersize=6, label=f'ブロック {block+1}')
    
    ax1.set_xlabel('Global Generation Step')
    ax1.set_ylabel('Average Confidence')
    ax1.set_title('Confidence Score Over Generation Steps (by Block)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 各ステップで明かされたトークン数
    bars = ax2.bar(global_steps, tokens_revealed, alpha=0.7)
    
    # ブロックごとに色分け
    for i, (bar, block) in enumerate(zip(bars, blocks)):
        bar.set_color(colors[block])
    
    ax2.set_xlabel('Global Generation Step')
    ax2.set_ylabel('Tokens Revealed')
    ax2.set_title('Number of Tokens Revealed per Step')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計情報
    print(f"平均信頼度: {np.mean(confidences):.3f}")
    print(f"信頼度の標準偏差: {np.std(confidences):.3f}")
    print(f"総トークン数: {sum(tokens_revealed)}")
    print(f"ブロック数: {len(unique_blocks)}")

def visualize_block_progression(history):
    """
    ブロック別の生成進捗を可視化
    """
    if not history:
        print("履歴データがありません")
        return
        
    print("\\n🔍 ブロック別生成進捗:")
    print("=" * 50)
    
    current_block = -1
    for entry in history:
        if entry['block'] != current_block:
            current_block = entry['block']
            print(f"\\n📦 ブロック {current_block + 1}:")
            
        step = entry['step']
        text = entry['text']
        num_revealed = entry['num_revealed']
        avg_conf = entry['avg_confidence']
        
        # [MASK]を視覚的に表現
        visual_text = text.replace('<|reserved_special_token_250|>', '[MASK]')
        
        print(f"  Step {step+1:2d}: +{num_revealed:2d} tokens (conf={avg_conf:.3f})")
        print(f"    状態: {visual_text[:80]}{'...' if len(visual_text) > 80 else ''}")

# 前回の数学問題の結果を分析
if 'math_history' in locals():
    print("数学問題の生成プロセス分析:")
    analyze_confidence_over_steps(math_history)
    visualize_block_progression(math_history[-10:])  # 最後の10ステップ
else:
    print("分析用のデータを生成中...")
    _, analysis_history = generate_llada(
        model, "Explain how photosynthesis works in plants.",
        steps=24, gen_length=80, block_length=20,
        temperature=0.0, cfg_scale=0.0, remasking='low_confidence'
    )
    analyze_confidence_over_steps(analysis_history)
    visualize_block_progression(analysis_history[-8:])

## 7. 自己回帰モデルとの比較

拡散モデルと自己回帰モデルの違いを理解するために、LLaDAの特徴を整理してみましょう。

In [ ]:
def compare_generation_strategies():
    """
    生成戦略の違いを視覚的に説明
    """
    print("📊 生成戦略の比較")
    print("=" * 60)
    
    # 自己回帰モデル（GPT風）のシミュレーション
    print("🔄 自己回帰モデル (GPT/LLaMA):")
    autoregressive_steps = [
        "The",
        "The capital",
        "The capital of",
        "The capital of Japan",
        "The capital of Japan is",
        "The capital of Japan is Tokyo"
    ]
    
    for i, step in enumerate(autoregressive_steps):
        print(f"  Step {i+1}: {step}")
    
    print("\n  特徴:")
    print("  ✓ 左から右へ順次生成")
    print("  ✓ 各トークンは過去のトークンのみ参照")
    print("  ✓ 高速（KV-cacheが使える）")
    print("  ✗ 後続の文脈を考慮できない")
    
    print("\n" + "=" * 60)
    
    # 拡散モデル（LLaDA）
    print("🌀 拡散モデル (LLaDA):")
    diffusion_steps = [
        "[MASK] [MASK] [MASK] [MASK] [MASK] [MASK]",
        "The [MASK] [MASK] Japan [MASK] [MASK]",
        "The capital [MASK] Japan [MASK] Tokyo",
        "The capital of Japan is Tokyo"
    ]
    
    for i, step in enumerate(diffusion_steps):
        print(f"  Step {i+1}: {step}")
    
    print("\n  特徴:")
    print("  ✓ 全体構造から詳細へ")
    print("  ✓ 双方向の文脈を考慮")
    print("  ✓ 反復的な品質改善")
    print("  ✗ 比較的低速（KV-cache使用不可）")
    
    print("\n" + "=" * 60)
    print("\n💡 主な違い:")
    print("1. 生成方向: 自己回帰は一方向、拡散は双方向")
    print("2. 品質向上: 拡散は複数ステップで反復改善")
    print("3. 計算効率: 自己回帰が高速、拡散は高品質")
    print("4. 制御性: 拡散はより細かい制御が可能")

compare_generation_strategies()

### 実際の性能比較実験

In [ ]:
def performance_analysis(prompts, steps_list=[8, 16, 32]):
    """
    異なるプロンプトとステップ数での性能分析
    """
    results = []
    
    for prompt in prompts:
        print(f"\n🔍 分析中: {prompt[:50]}...")
        
        for steps in steps_list:
            start_time = time.time()
            
            result, history = simple_generate(
                model, prompt,
                steps=steps, gen_length=40, temperature=0.0
            )
            
            generation_time = time.time() - start_time
            avg_confidence = np.mean([h['avg_confidence'] for h in history])
            
            results.append({
                'prompt': prompt,
                'steps': steps,
                'time': generation_time,
                'avg_confidence': avg_confidence,
                'result': result
            })
            
            print(f"  {steps:2d}ステップ: {generation_time:.2f}秒, 信頼度{avg_confidence:.3f}")
    
    return results

# 異なるタイプの質問で性能テスト
test_prompts = [
    "What is 15 + 27?",  # 簡単な計算
    "Explain the water cycle.",  # 説明文
    "Write a short poem about spring.",  # 創作
]

perf_results = performance_analysis(test_prompts, [8, 16, 24])

In [ ]:
# 結果の可視化
def visualize_performance_results(results):
    # データの整理
    steps_data = {}
    for result in results:
        steps = result['steps']
        if steps not in steps_data:
            steps_data[steps] = {'times': [], 'confidences': []}
        steps_data[steps]['times'].append(result['time'])
        steps_data[steps]['confidences'].append(result['avg_confidence'])
    
    # グラフ作成
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    steps_list = sorted(steps_data.keys())
    avg_times = [np.mean(steps_data[s]['times']) for s in steps_list]
    avg_confidences = [np.mean(steps_data[s]['confidences']) for s in steps_list]
    
    # 生成時間
    ax1.bar(steps_list, avg_times, color='skyblue', alpha=0.7)
    ax1.set_xlabel('Number of Steps')
    ax1.set_ylabel('Average Generation Time (seconds)')
    ax1.set_title('Generation Time vs Steps')
    ax1.grid(True, alpha=0.3)
    
    # 平均信頼度
    ax2.bar(steps_list, avg_confidences, color='lightcoral', alpha=0.7)
    ax2.set_xlabel('Number of Steps')
    ax2.set_ylabel('Average Confidence Score')
    ax2.set_title('Confidence vs Steps')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📈 性能分析結果:")
    for steps in steps_list:
        avg_time = np.mean(steps_data[steps]['times'])
        avg_conf = np.mean(steps_data[steps]['confidences'])
        print(f"  {steps:2d}ステップ: 平均{avg_time:.2f}秒, 信頼度{avg_conf:.3f}")

visualize_performance_results(perf_results)

## 8. まとめと発展的な実験

### 学んだこと

このノートブックを通じて、以下のような拡散言語モデルの特徴を学びました：

1. **拡散プロセス**: [MASK]トークンから始まり、段階的に実際のトークンを明かしていく
2. **信頼度ベース選択**: モデルの予測信頼度に基づいて、どのトークンを確定するかを決定
3. **双方向文脈**: 従来の自己回帰モデルと異なり、全体の文脈を考慮できる
4. **パラメータ制御**: ステップ数、温度などで生成の品質と多様性を調整可能

### 発展的な実験

以下のような実験をさらに試してみることができます：

In [ ]:
def advanced_experiments():
    """
    発展的な実験のアイデアを提示
    """
    experiments = [
        {
            "title": "🎯 制約付き生成",
            "description": "特定の単語を特定の位置に配置した生成",
            "example": "位置3に'beautiful'、位置7に'sunset'を指定"
        },
        {
            "title": "🔄 再マスキング戦略",
            "description": "'low_confidence'と'random'の違いを比較",
            "example": "同じプロンプトで戦略を変えて結果を比較"
        },
        {
            "title": "📊 長文生成",
            "description": "長いテキストでの拡散プロセスの観察",
            "example": "200トークン以上の物語や記事の生成"
        },
        {
            "title": "🌡️ 温度スケジューリング",
            "description": "ステップごとに温度を変化させる",
            "example": "初期は高温度、後期は低温度で安定化"
        },
        {
            "title": "🎨 創作性評価",
            "description": "詩や物語の創作での拡散の効果",
            "example": "同じテーマで複数回生成して多様性を測定"
        }
    ]
    
    print("🚀 発展的な実験アイデア")
    print("=" * 50)
    
    for i, exp in enumerate(experiments, 1):
        print(f"\n{i}. {exp['title']}")
        print(f"   概要: {exp['description']}")
        print(f"   例: {exp['example']}")
    
    print("\n" + "=" * 50)
    print("💡 実験のヒント:")
    print("- 異なるドメイン（数学、科学、文学）での性能を比較")
    print("- 生成品質の定量的評価指標を考案")
    print("- 他の拡散モデルとの比較研究")
    print("- リアルタイム可視化ツールの開発")

advanced_experiments()

### 簡単な発展実験: 創作性の比較

In [ ]:
def creativity_experiment():
    """
    創作的なタスクでの拡散モデルの特徴を調べる
    """
    creative_prompt = "Write a creative story about a robot discovering emotions."
    
    print("🎨 創作性実験")
    print("プロンプト:", creative_prompt)
    print("=" * 60)
    
    # 異なる設定で生成
    settings = [
        {
            "steps": 16, "block_length": 40, "temperature": 0.0, 
            "cfg_scale": 0.0, "remasking": "low_confidence",
            "name": "決定論的・高速"
        },
        {
            "steps": 32, "block_length": 20, "temperature": 0.0, 
            "cfg_scale": 0.0, "remasking": "low_confidence",
            "name": "決定論的・高品質"
        },
        {
            "steps": 24, "block_length": 30, "temperature": 0.4, 
            "cfg_scale": 0.0, "remasking": "low_confidence",
            "name": "創造的・中品質"
        },
        {
            "steps": 20, "block_length": 25, "temperature": 0.2, 
            "cfg_scale": 1.5, "remasking": "low_confidence",
            "name": "CFG強化・創造的"
        },
        {
            "steps": 18, "block_length": 35, "temperature": 0.0, 
            "cfg_scale": 0.0, "remasking": "random",
            "name": "ランダム再マスク"
        }
    ]
    
    results = []
    
    for setting in settings:
        print(f"\\n📝 {setting['name']}:")
        print(f"  設定: steps={setting['steps']}, block={setting['block_length']}, "
              f"temp={setting['temperature']}, cfg={setting['cfg_scale']}, "
              f"remasking={setting['remasking']}")
        
        start_time = time.time()
        result, history = generate_llada(
            model, creative_prompt,
            steps=setting['steps'],
            gen_length=80,
            block_length=setting['block_length'],
            temperature=setting['temperature'],
            cfg_scale=setting['cfg_scale'],
            remasking=setting['remasking']
        )
        generation_time = time.time() - start_time
        
        avg_confidence = np.mean([h['avg_confidence'] for h in history])
        
        results.append({
            'setting': setting['name'],
            'text': result,
            'time': generation_time,
            'avg_confidence': avg_confidence,
            'parameters': setting
        })
        
        print(f"  結果: {result}")
        print(f"  時間: {generation_time:.2f}秒, 平均信頼度: {avg_confidence:.3f}")
    
    print("\\n" + "=" * 60)
    print("🔍 分析:")
    print("- 温度0.0では一貫した結果が得られる")
    print("- ステップ数を増やすことで詳細度が向上")
    print("- 適度な温度は創造性を高める可能性")
    print("- CFGは条件付き生成の品質を向上")
    print("- ランダム再マスクは予期しない創造性を提供")
    
    return results

def compare_remasking_strategies():
    """
    再マスキング戦略の比較実験
    """
    prompt = "Describe the process of making coffee step by step."
    
    print("\\n🔄 再マスキング戦略比較")
    print("プロンプト:", prompt)
    print("=" * 60)
    
    strategies = ['low_confidence', 'random']
    results = {}
    
    for strategy in strategies:
        print(f"\\n🎯 戦略: {strategy}")
        
        # 同じ設定で複数回実行して安定性を確認
        runs = []
        for run in range(3):
            result, history = generate_llada(
                model, prompt,
                steps=20, gen_length=60, block_length=20,
                temperature=0.0, cfg_scale=0.0, remasking=strategy
            )
            
            avg_confidence = np.mean([h['avg_confidence'] for h in history])
            runs.append({
                'run': run + 1,
                'result': result,
                'avg_confidence': avg_confidence
            })
            
            print(f"  実行 {run + 1}: 信頼度 {avg_confidence:.3f}")
            print(f"    結果: {result[:60]}{'...' if len(result) > 60 else ''}")
        
        results[strategy] = runs
    
    print("\\n" + "=" * 60)
    print("📊 戦略比較結果:")
    for strategy, runs in results.items():
        avg_conf = np.mean([r['avg_confidence'] for r in runs])
        std_conf = np.std([r['avg_confidence'] for r in runs])
        print(f"{strategy}: 平均信頼度 {avg_conf:.3f} ± {std_conf:.3f}")
    
    return results

# 創作実験を実行
creativity_results = creativity_experiment()

# 再マスキング戦略比較
remasking_results = compare_remasking_strategies()

### 最終チャレンジ: オリジナル実験

In [ ]:
# あなた自身の実験を試してみましょう！

print("🎯 チャレンジ: あなた自身の実験")
print("=" * 40)
print("完全なLLaDAの機能を使って自由に実験してみてください:")
print()
print("📝 プロンプトの例:")
print('- "Explain quantum physics in simple terms."')
print('- "Write a dialogue between two AIs."')
print('- "Create a recipe for happiness."')
print('- "Describe a day in the life of a photon."')
print()
print("⚙️ 利用可能なパラメータ:")
print("- steps: 8~64 (推奨: 16, 24, 32)")
print("- gen_length: 32~128 (推奨: 48, 64, 80)")
print("- block_length: 8~gen_length (推奨: 16, 24, 32)")
print("- temperature: 0.0~1.0 (推奨: 0.0, 0.3, 0.5)")
print("- cfg_scale: 0.0~3.0 (推奨: 0.0, 1.0, 2.0)")
print("- remasking: 'low_confidence' または 'random'")
print()
print("🚀 新機能:")
print("✓ セミ自己回帰生成（ブロック処理）")
print("✓ 分類器フリーガイダンス（CFG）")
print("✓ 再マスキング戦略選択")
print("✓ 詳細な生成プロセス追跡")
print()
print("例:")
print('your_result, your_history = generate_llada(')
print('    model, "あなたのプロンプト",')
print('    steps=24, gen_length=64, block_length=16,')
print('    temperature=0.2, cfg_scale=1.0, remasking="low_confidence"')
print(')')
print('visualize_block_progression(your_history)')
print('analyze_confidence_over_steps(your_history)')
print()
print("💡 実験アイデア:")
print("1. CFGスケール（0.0, 1.0, 2.0）の影響を比較")
print("2. ブロックサイズが生成品質に与える影響")
print("3. 創作的なタスクでの温度とCFGの組み合わせ")
print("4. 科学的説明vs創作的文章での最適設定")
print("5. 再マスキング戦略の使い分け")

# ここにあなたの実験コードを書いてください！
# 例:
# my_prompt = "Write a short poem about the beauty of mathematics."
# my_result, my_history = generate_llada(
#     model, my_prompt,
#     steps=20, gen_length=60, block_length=20,
#     temperature=0.3, cfg_scale=1.2, remasking="low_confidence"
# )
# print(f"結果: {my_result}")
# visualize_block_progression(my_history[-5:])  # 最後の5ステップを表示

## 🎓 学習のまとめ

### LLaDA拡散言語モデルで学んだ重要なポイント

1. **拡散プロセスの理解**
   - 全てのトークンを[MASK]で初期化
   - 信頼度に基づく段階的な明かし
   - 双方向の文脈考慮

2. **自己回帰との違い**
   - 生成方向: 一方向 vs 全方向
   - 品質改善: 一回 vs 反復的
   - 制御性: 限定的 vs 柔軟

3. **パラメータの影響**
   - ステップ数: 品質と計算時間のトレードオフ
   - 温度: 決定論的 vs 創造的
   - 生成長: 応答の詳細度

4. **実用的な洞察**
   - 数学的な問題: 低温度、多ステップが効果的
   - 創作的なタスク: 中程度の温度で多様性向上
   - 短い応答: 少ないステップでも十分

### 次のステップ

- 他の拡散言語モデルとの比較
- 専門分野（医療、法律、技術）での性能評価
- リアルタイム応用の可能性探索
- ファインチューニング手法の研究

**おめでとうございます！** 拡散言語モデルの基礎を理解し、実際にLLaDAを使った実験を完了しました。🎉